In [1]:
%pip install transformers pyctcdecode datasets torch librosa

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install https://github.com/kpu/kenlm/archive/master.zip

  Using cached https://github.com/kpu/kenlm/archive/master.zip
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


In [3]:
import argparse
import sys
import re
from typing import Dict

import torch
from datasets import Audio, Dataset, load_dataset, load_metric

from pyctcdecode import BeamSearchDecoderCTC
from transformers import AutoFeatureExtractor, AutoTokenizer, pipeline
import transformers
import librosa

c:\Users\Krist\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# # recording output
# import sounddevice as sd
# from scipy.io.wavfile import write
# import wavio as wv
 
# # Sampling frequency
# freq = 44100
  
# # Recording duration
# duration = 5
  
# # Start recorder with the given values 
# # of duration and sample frequency
# recording = sd.rec(int(duration * freq), 
#                    samplerate=freq, channels=2)

# print("start recording")  
# # Record audio for the given number of seconds
# sd.wait()
# print("end recording")

  
# # 2 ways of saving:
# write("output.wav", freq, recording)

In [4]:
# model id
model_id = 'FremyCompany/xls-r-2b-nl-v2_lm-5gram-os'

# load processor
feature_extractor = AutoFeatureExtractor.from_pretrained(model_id)
sampling_rate = 16000 # I recommend this, much faster than 44.1kHz without sacrificing much quality

# load eval pipeline
device = 0 if torch.cuda.is_available() else -1 # Running on GPU (0) if pytorch detects that you have it, otherwise CPU (-1)

# individually initializing all the components of speech recognition
config = transformers.PretrainedConfig.from_pretrained(model_id)
model = transformers.Wav2Vec2ForCTC.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)
processor = transformers.AutoProcessor.from_pretrained(model_id)
language_model = BeamSearchDecoderCTC.model_container[processor.decoder._model_key]._kenlm_model

# initializing the pipeline 
asr = pipeline("automatic-speech-recognition", config=config, model=model, tokenizer=tokenizer, feature_extractor=feature_extractor, decoder=processor.decoder, device=device)


You are using a model of type wav2vec2 to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
c:\Users\Krist\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\utils\_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'snapshot_download': allow_regex. Will not be supported from version '0.12'.

Please use `allow_patterns` and `ignore_patterns` instead.
  warnings.warn(message, FutureWarning)
Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 181.72it/s]


In [8]:
# resample audio
wav_file = "output.wav" # Or some other .wav audio file
sound_array, _ = librosa.load(wav_file, sr = sampling_rate) # Loading audio with librosa to a 'sound array', so just a long list of 16000 numbers per second 

# map function to decode audio
def map_to_pred(sound_array, chunk_length_s = 10, stride_length_s = 2):
    prediction = asr(sound_array, chunk_length_s = chunk_length_s, stride_length_s = stride_length_s)
    text = prediction["text"] # Get only the text from the prediction, disregard other info
    return text


In [9]:
print(map_to_pred(sound_array))

dit is eli
